In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords             #불용어
from urllib.request import urlretrieve
import zipfile
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

In [2]:
import numpy as np
import gensim
from urllib.request import urlretrieve, urlopen
import gzip
import zipfile
from gensim.models.word2vec import Word2Vec
import pandas as pd

In [19]:
import re

In [20]:
data=pd.read_csv("D:/test.csv", engine='python')
data.head()

,article_id,article_text,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,1,"우리는 많은 시간 인터넷을 이용하고 있지만, 이는 인터넷 회선을 통해 인터넷 서비스...",NaN,NaN,NaN
1,2,우리가 논의를 좀 더 전개하기 전에 먼저 이론적인 것 잠깐만 짚고 넘어가겠습니다. ...,NaN,NaN,NaN
2,3,GAN은 콘텐츠 생산 산업에 많은 변화를 가져올 것으로 보인다. 그러나 부정적인 측...,NaN,NaN,NaN
3,4,서양미술 혁신의 첫번째로 르네상스 미술 이야기부터 하겠습니다.\n한마디로 인간성의 ...,NaN,NaN,NaN
4,5,힘은 크기와 방향을 갖는 벡터양으로서 힘의 크기를 나타내는 SI 단위는 뉴턴(N)이...,NaN,NaN,NaN


In [21]:
data=data[['article_text']]

stop_words=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def preprocess_sentence(sentence):
    sentence=[re.sub(r'[^가-힣\s]','',word) for word in sentence]
    return [word for word in sentence if word not in stop_words and word]

def preprocess_sentences(sentences):
    return [preprocess_sentence(sentence) for sentence in sentences]
#a-zA-Z, 즉 영어는 포함 안되게 했음

In [22]:
data

,article_text
0,"우리는 많은 시간 인터넷을 이용하고 있지만, 이는 인터넷 회선을 통해 인터넷 서비스..."
1,우리가 논의를 좀 더 전개하기 전에 먼저 이론적인 것 잠깐만 짚고 넘어가겠습니다. ...
2,GAN은 콘텐츠 생산 산업에 많은 변화를 가져올 것으로 보인다. 그러나 부정적인 측...
3,서양미술 혁신의 첫번째로 르네상스 미술 이야기부터 하겠습니다.\n한마디로 인간성의 ...
4,힘은 크기와 방향을 갖는 벡터양으로서 힘의 크기를 나타내는 SI 단위는 뉴턴(N)이...


In [30]:
data=data[['article_text']]
data['sentences']=data['article_text'].apply(sent_tokenize)
data['sentences'][0]

D:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


['우리는 많은 시간 인터넷을 이용하고 있지만, 이는 인터넷 회선을 통해 인터넷 서비스를 이용하는 것일 뿐, 내 컴퓨터 안의 파일이 인터넷에 연결되어 있는 것은 아닙니다.',
 'HTML로 웹사이트를 만들고 그 내용을 다른 사람들이 볼 수 있도록 하려면 HTML로 만든 웹 문서를 모두 서버 컴퓨터로 옮겨야 합니다.',
 '서버(server)컴퓨터란 전용선을 통해 인터넷에 직접 연결되어 있는 컴퓨터를 가리키는데, 24시간 인터넷에 연결되어있고 서버 컴퓨터 접속 주소만 알면 누구나 서버 컴퓨터의 내용을 볼 수 있습니다.',
 '인터넷 회선을 통해 서버 컴퓨터에 접속하는 사용자 컴퓨터를 클라이언트(client)컴퓨터라고 합니다.',
 '서버 정보를 가져와 보여주는 것은 사용자 컴퓨터 안의 웹 브라우저이기 때문에 좁은 의미로 웹브라우저를 클라이언트라고도 합니다.',
 '웹 디자이너나 웹 개발자들은 자신이 제작한 최신 웹사이트를 항상 서버 컴퓨터에 업로드해 놓기 때문에 사용자들은 자신의 위치에 상관없이 어디에서나 인터넷에 접속해서 해당 웹사이트의 내용을 볼 수 있습니다.',
 '개인은 웹 서버를 마련하기 어렵기 때문에 서버의 일부 공간을 매달 혹은 몇 년마다 일정 금액을 내고 사용하는 서비스를 이용합니다.',
 "이것을 '서버 호스팅 서비스' 혹은 '웹 호스팅 서비스'라고 하는데, 개인 웹사이트를 운영하는 사람들은 대부분 이런 호스팅 서비스를 이용합니다.",
 '호스팅 서비스는 어떤 서버를 이용하느냐에 따라 윈도우 서버 호스팅과 리눅스 서버 호스팅으로 나뉘는데, 윈도우 서버에서는 ASP나 ASAP.NET 프로그래밍 언어를 사용하고, 리눅스 서버에서는 PHP 프로그래밍 언어를 사용하며 좀 더 대중적이고 저렴합니다.']

In [24]:
kovec=Word2Vec.load("D:\ko.bin")
#kovec=gensim.models.KeyedVectors.load_word2vec_format('D:\ko.bin',binary=True)

In [25]:
model=Word2Vec(data,sg=1,size=100,window=3,min_count=3,workers=4)

In [26]:
kovec.wv.most_similar("사람")

[('젊은이', 0.6494427919387817),
 ('여인', 0.6287257671356201),
 ('백성', 0.6063710451126099),
 ('포졸', 0.6043275594711304),
 ('죄인', 0.5960500836372375),
 ('선비', 0.5868039131164551),
 ('부녀자', 0.5654411315917969),
 ('죄수', 0.5639811754226685),
 ('구경꾼', 0.5620019435882568),
 ('손님', 0.5589558482170105)]

In [27]:
kovec.wv['사람']

array([ 0.7412252 , -1.1187236 ,  2.0681474 ,  1.05749   , -1.3211275 ,
       -2.414994  , -1.133681  ,  1.5452526 , -0.4543698 ,  0.02380616,
        0.12370981, -0.8839078 , -2.9051683 , -0.34940982,  1.3418512 ,
        2.8225965 ,  1.0794274 ,  0.4287113 , -1.9613483 ,  0.4262147 ,
       -0.40650806, -0.34994036, -0.8416261 ,  0.52574843, -0.29543158,
       -0.5593075 ,  0.84687376,  0.0889315 , -2.5240638 ,  0.46828857,
        2.8982353 , -1.6508341 , -0.37446976, -2.2825975 ,  0.7083552 ,
       -0.59711653,  0.2839199 , -1.0247895 ,  2.3036754 ,  0.45918384,
       -0.83396995,  0.85293204,  0.972802  , -0.5457918 , -0.93798906,
       -0.8009529 ,  2.160266  ,  0.10052549, -0.16650781, -0.07592812,
       -0.45869842,  1.3352628 ,  2.3284414 ,  0.7132327 , -2.4690466 ,
        0.05213251,  0.38745207, -0.2773064 , -0.67375   , -0.59059584,
        1.0310122 ,  1.9923794 , -0.94680166, -1.3167765 , -1.4717319 ,
        0.97447944,  0.8103551 ,  0.8915873 ,  0.7474684 , -0.00

In [28]:
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [29]:
## 형태소 단위로 Tokenize
from konlpy.tag import Okt  
okt=Okt()

def tokenization(sentences):
    temp=[]
    for sentence in sentences:
        temp_X=okt.morphs(sentence,stem=True)
        temp_X=[word for word in temp_X]
        temp.append(temp_X)
    return temp


data['tokenized_data']=data['sentences'].apply(tokenization)
    
data['tokenized_data']=data['tokenized_data'].apply(preprocess_sentences)
    
data

,article_text,sentences,tokenized_data
0,"우리는 많은 시간 인터넷을 이용하고 있지만, 이는 인터넷 회선을 통해 인터넷 서비스...","[우리는 많은 시간 인터넷을 이용하고 있지만, 이는 인터넷 회선을 통해 인터넷 서비...","[[우리, 많다, 시간, 인터넷, 을, 이용, 하고, 있다, 이다, 인터넷, 회선,..."
1,우리가 논의를 좀 더 전개하기 전에 먼저 이론적인 것 잠깐만 짚고 넘어가겠습니다. ...,[우리가 논의를 좀 더 전개하기 전에 먼저 이론적인 것 잠깐만 짚고 넘어가겠습니다....,"[[우리, 논의, 더, 전개, 전, 먼저, 이론, 적, 인, 것, 잠깐, 만, 짚다..."
2,GAN은 콘텐츠 생산 산업에 많은 변화를 가져올 것으로 보인다. 그러나 부정적인 측...,"[GAN은 콘텐츠 생산 산업에 많은 변화를 가져올 것으로 보인다., 그러나 부정적인...","[[콘텐츠, 생산, 산업, 많다, 변화, 가져오다, 것, 보이다], [그러나, 부정..."
3,서양미술 혁신의 첫번째로 르네상스 미술 이야기부터 하겠습니다.\n한마디로 인간성의 ...,"[서양미술 혁신의 첫번째로 르네상스 미술 이야기부터 하겠습니다., 한마디로 인간성의...","[[서양미술, 혁신, 첫, 번째, 로, 르네상스, 미술, 이야기, 부터], [한마디..."
4,힘은 크기와 방향을 갖는 벡터양으로서 힘의 크기를 나타내는 SI 단위는 뉴턴(N)이...,[힘은 크기와 방향을 갖는 벡터양으로서 힘의 크기를 나타내는 SI 단위는 뉴턴(N)...,"[[힘, 크기, 방향, 을, 갖다, 벡터, 양, 으로서, 힘, 크기, 나타내다, 단..."


In [ ]:
data['tokenized_data']

In [ ]:
embedding_dim=200
zero_vector=np.zeros(embedding_dim)

In [ ]:
def calculate_sentence_vector(sentence):
    if len(sentence) != 0:
        sum=0
        for word in sentence:
            try: 
                sum+=kovec.wv[word]
            except Exception:
                sum+=zero_vector
        return sum/len(sentence)
    else:
        return zero_vector

In [ ]:
def sentences_to_vectors(sentences):
    return [calculate_sentence_vector(sentence) for sentence in sentences]

In [ ]:
data['SentenceEmbedding']=data['tokenized_data'].apply(sentences_to_vectors)
data[['SentenceEmbedding']]

In [ ]:
def similarity_matrix(sentence_embedding):
    sim_mat=np.zeros([len(sentence_embedding),len(sentence_embedding)])
    for i in range(len(sentence_embedding)):
        for j in range(len(sentence_embedding)):
            sim_mat[i][j]=cosine_similarity(sentence_embedding[i].reshape(1,embedding_dim),
                                           sentence_embedding[j].reshape(1,embedding_dim))[0,0]
    return sim_mat

In [ ]:
data['SimMatrix']=data['SentenceEmbedding'].apply(similarity_matrix)
data['SimMatrix']

In [ ]:
print('두번째 샘플의 문장 개수 : ',len(data['tokenized_data'][0]))
print('두번째 샘플의 문장 벡터가 모인 문장 행렬의 크기(shape) : ',np.shape(data['SentenceEmbedding'][0]))
print('두번쨰 샘플의 유사도 행렬의 크기(shape) : ',data['SimMatrix'][0].shape)

In [ ]:
data

In [ ]:
def draw_graphs(sim_matrix):
    nx_graph=nx.from_numpy_array(sim_matrix)
    plt.figure(figsize=(10,10))
    pos=nx.spring_layout(nx_graph)
    nx.draw(nx_graph,with_labels=True,font_weight='bold')
    nx.draw_networkx_edge_labels(nx_graph,pos,font_color='red')
    plt.show()

In [ ]:
draw_graphs(data['SimMatrix'][0])

In [ ]:
def calculate_score(sim_matrix):
    nx_graph=nx.from_numpy_array(sim_matrix)
    scores=nx.pagerank_numpy(nx_graph)
    return scores

In [ ]:
data['score']=data['SimMatrix'].apply(calculate_score)
data[['SimMatrix','score']]

In [ ]:
data['score'][0]

In [ ]:
def ranked_sentences(sentences,scores,n=3):
    top_scores=sorted(((scores[i],s)
                      for i,s in enumerate(sentences)),
                     reverse=True)
    top_n_sentences=[sentence for score,sentence in top_scores[:n]]
    return " ".join(top_n_sentences)

In [ ]:
data

In [ ]:
data['summary'] = data.apply(lambda x:
                            ranked_sentences(x.sentences, x.score),axis=1)

In [ ]:
for i in range(0,len(data)):
    print(i+1,'번 문서')
    print('원문 : ',data.loc[i].article_text)
    print(' ')
    print('요약 : ',data.loc[i].summary)
    print('')